# MODEL 1 SUCCESS

#Library

In [155]:
!pip install spacy
!pip show spacy

Name: spacy
Version: 3.7.4
Summary: Industrial-strength Natural Language Processing (NLP) in Python
Home-page: https://spacy.io
Author: Explosion
Author-email: contact@explosion.ai
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: catalogue, cymem, jinja2, langcodes, murmurhash, numpy, packaging, preshed, pydantic, requests, setuptools, smart-open, spacy-legacy, spacy-loggers, srsly, thinc, tqdm, typer, wasabi, weasel
Required-by: en-core-web-sm, fastai


In [156]:
!pip install path

In [157]:
#Library
import spacy
from spacy import load, displacy
from spacy.util import minibatch, compounding
from pathlib import Path
import pickle
import random

In [158]:
with open('train_data.pickle', 'rb') as f:
    TRAIN_DATA = pickle.load(f)
len(TRAIN_DATA)

3649

In [159]:
TRAIN_DATA

[('Pengamat politik dari Universitas Gadjah Mada Arie Sudjito menilai, keinginan Ketua Umum Partai Golkar Aburizal Bakrie untuk maju kembali sebagai ketua umum merupakan pemaksaan kehendak.',
  {'entities': [(22, 45, 'ORGANIZATION'),
    (46, 58, 'PERSON'),
    (89, 102, 'ORGANIZATION'),
    (103, 118, 'PERSON')]}),
 ('Menurut dia, ada kesan bahwa Aburizal menggunakan segala cara untuk memuluskan jalannya kembali menduduki Golkar 1.',
  {'entities': [(29, 37, 'PERSON'), (106, 112, 'ORGANIZATION')]}),
 ('Hal ini, kata Arie berpotensi menimbukan perpecahan di kalangan kader Golkar di daerah.',
  {'entities': [(14, 18, 'PERSON'), (70, 76, 'PERSON')]}),
 ('"Tentu ini akan merugikan Golkar karena Golkar akan pecah dan konflik di daerah dipastikan akan terjadi," ujar Arie saat dihubungi Kompas.com, Selasa (2/12/2014)',
  {'entities': [(26, 32, 'ORGANIZATION'),
    (40, 46, 'ORGANIZATION'),
    (111, 115, 'PERSON')]}),
 ('Arie mengatakan, sikap Aburizal tersebut ditentang oleh sebagian kalang

In [160]:
#Load Blank model
nlp=spacy.blank("id")
#pipeline NER
ner=nlp.create_pipe('ner')
nlp.add_pipe('ner')

### ERROR

In [45]:
#Label entitas
for _, annotations in TRAIN_DATA:
  for ent in annotations.get('entities'):
    ner.add_label(ent[2])

#ksemampingkan pipeline selain NER
other_pipes= [pipe for pipe in nlp.pipe_names if pipe != 'ner']

#Train Model
n_iter = 5
with nlp.disable_pipes(other_pipes):
  optimizer = nlp.begin_training()
  for iteration in range(n_iter):
      random.shuffle(TRAIN_DATA)
      losses = {}
      batches = minibatch(TRAIN_DATA, size=compounding(4.0,32.0,1.001))
      for batch in batches:
          texts, annotations = zip(*batch)
          nlp.update(
                    texts,
                    annotations,
                    drop=0.5,
                    sgd=optimizer,
                    losses=losses)
      print("Losses at iteration {}".format(iteration),losses)


ValueError: [E989] `nlp.update()` was called with two positional arguments. This may be due to a backwards-incompatible change to the format of the training data in spaCy 3.0 onwards. The 'update' function should now be called with a batch of Example objects, instead of `(text, annotation)` tuples. 

## BENER

In [161]:
import spacy
from spacy.training.example import Example
from spacy.util import minibatch, compounding
import random


# Load blank English model
nlp = spacy.blank("id")
# Create the NER pipeline component and add it to the pipeline
ner = nlp.create_pipe("ner")
nlp.add_pipe("ner", last=True)

# Add labels to the NER component
for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

# Exclude other pipes from the training loop
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

# Train the model
n_iter = 10
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.begin_training()
    for iteration in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            examples = [Example.from_dict(nlp.make_doc(text), annotation) for text, annotation in zip(texts, annotations)]
            nlp.update(
                examples,
                drop=0.5,
                sgd=optimizer,
                losses=losses
            ) #sgd=optimizer default
        print("Losses at iteration {}".format(iteration), losses)



/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "JAKARTA - Wakil Ketua Fraksi Partai Keadilan Sejah..." with entities "[[10, 56, 'JOBTITLE'], [68, 76, 'PERSON'], [182, 1...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Tim gabungan Kemendagri, KASN dan Pemprov Jawa Tim..." with entities "[[14, 24, 'ORGANIZATION'], [27, 31, 'ORGANIZATION'...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be 

Losses at iteration 0 {'ner': 7516.624429312446}


/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "" Dari hari Selasa (12/1/2021), ini kebetulan untu..." with entities "[[11, 29, 'TIME'], [92, 112, 'FACILITY'], [151, 15...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Kebakaran terjadi di kawasan permukiman padat pend..." with entities "[[0, 9, 'ACCIDENT'], [58, 114, 'LOCATION'], [116, ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be 

Losses at iteration 1 {'ner': 5144.429618113705}


/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "" Kalau saya dan bu @ smindrawati sudah siap divak..." with entities "[[19, 37, 'SOCIALMEDIA_ACCOUNT'], [74, 79, 'PERSON...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Nama Listyo merupakan bagian lima nama yang diusul..." with entities "[[5, 11, 'PERSON'], [54, 80, 'ORGANIZATION'], [100...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be 

...

Losses at iteration 7 {'ner': 3227.540439848405}


/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Komisi Hukum atau Komisi III Dewan Perwakilan Raky..." with entities "[[0, 12, 'ORGANIZATION'], [18, 52, 'ORGANIZATION']...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "" Menhan AS menawarkan segala dukungan yang dibutu..." with entities "[[1, 10, 'JOBTITLE'], [54, 67, 'COUNTRY'], [110, 1...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be 

Losses at iteration 8 {'ner': 2955.3278439790224}


/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Ruas jalan provinsi yang menghubungkan Banjarnegar..." with entities "[[40, 74, 'LOCATION'], [149, 165, 'TIME']]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Komisi Hukum atau Komisi III Dewan Perwakilan Raky..." with entities "[[0, 12, 'ORGANIZATION'], [18, 52, 'ORGANIZATION']...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in 

Losses at iteration 9 {'ner': 2818.1755736351834}


In [162]:
#Model save
output_dir= Path('nlp_id_checkpoint_2024_06_11')
nlp.to_disk(output_dir)
print("saved model to", output_dir)

saved model to nlp_id_checkpoint_2024_06_11


In [163]:
#Load Existing model
output_dir= '/content/nlp_id_checkpoint_2024_06_11'
print("Loading from", output_dir)
nlp = spacy.load(output_dir)


Loading from /content/nlp_id_checkpoint_2024_06_11


In [167]:
test= input("Input teks mau dilakukan NER:  ")  #dri teks berita: https://kemenag.go.id/internasional/kjri-jeddah-pengguna-visa-non-haji-lebih-baik-segera-pulang-ke-indonesia-AZUF7
doc= nlp(test)
print("Teks yang diinputkan>> ", test)
print("Entitas", [(ent.text, ent.label_) for ent in doc.ents])

Input teks mau dilakukan NER:  Jeddah (Kemenag) --- Konsul Jenderal Republik Indonesia di Jeddah, Yusron B Ambary kembali mengingatkan kepada jemaah pengguna visa non haji agar tidak memaksakan diri untuk berhaji. Mengingat, sanksi yang telah ditetapkan Pemerintah Arab Saudi bagi pelanggar yang tertangkap adalah deportasi dan denda 10.000 riyal.  "Arahan kami, sedapat mungkin untuk tidak berangkat haji, lebih baik segera pulang ke Indonesia," ungkapnya saat di Bandara Jeddah (8/6/2024).  Kepada jemaah pengguna visa non haji yang saat ini sudah berada di Makkah, Yusron berpesan agar jemaah segera mengambil langkah untuk pulang dan keluar dari Kota Makkah.  "Karena nanti akan terjadi pengetatan lebih lanjut di area Makkah dan sekitarnya, namun bagi jemaah yang mau pulang segera keluar dari Kota Makkah dan tidak memaksakan diri untuk berhaji," tegasnya.  Dikatakan Yusron, pihaknya akan mengurus kepulangan jemaah pengguna visa non haji, jika mereka nanti ingin pulang sebagaimana kasus sebe